In [1]:
# %%
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import chardet

# %%
# Deteksi encoding file CSV
with open('../data/data.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))  # Baca 10KB pertama
    print(result)

# Baca data dengan encoding hasil deteksi
df = pd.read_csv('../data/data.csv', encoding=result['encoding'], header=0)
print(df.head())
print(df.columns)
df.head()

# %%
print(df.columns)

# %%
# Fitur (Tweet) dan target (semua label multilabel)
X = df['Tweet']
y = df[['HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race', 
        'HS_Physical', 'HS_Gender', 'HS_Other', 'HS_Weak', 'HS_Moderate', 'HS_Strong']]

# %%
# Daftar stopword bahasa Indonesia manual
stopwords_indonesia = [
    'yang', 'dan', 'di', 'ke', 'dari', 'ini', 'itu', 'untuk', 'pada', 'dengan', 
    'adalah', 'atau', 'juga', 'sebagai', 'karena', 'oleh', 'tidak', 'sudah',
    'saja', 'maka', 'lagi', 'bagi', 'namun', 'setelah', 'sebelum', 'hingga', 
    'tersebut', 'setiap', 'kami', 'kita', 'mereka', 'aku', 'saya', 'anda', 
    'engkau', 'dia', 'ia', 'kalian', 'jadi', 'apakah', 'dimana', 'kapan',
    'mengapa', 'bagaimana', 'bahwa', 'tentang'
]

# TF-IDF Vectorizer dengan stopword bahasa Indonesia manual
vectorizer = TfidfVectorizer(stop_words=stopwords_indonesia, max_features=5000)

# Transform X
X_tfidf = vectorizer.fit_transform(X)

# %%
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# %%
# Model Random Forest multilabel
rf = RandomForestClassifier(n_estimators=100, random_state=42)
multi_rf = MultiOutputClassifier(rf)
multi_rf.fit(X_train, y_train)

# %%
# Prediksi
y_pred = multi_rf.predict(X_test)

# %%
# Evaluasi per label
for i, column in enumerate(y.columns):
    print(f"\n=== Evaluasi untuk label: {column} ===")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))
    print(f"Akurasi: {accuracy_score(y_test.iloc[:, i], y_pred[:, i]):.4f}")


{'encoding': 'ISO-8859-1', 'confidence': 0.728467, 'language': ''}
                                               Tweet  HS  Abusive  \
0  - disaat semua cowok berusaha melacak perhatia...   1        1   
1  RT USER: USER siapa yang telat ngasih tau elu?...   0        1   
2  41. Kadang aku berfikir, kenapa aku tetap perc...   0        0   
3  USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...   0        0   
4  USER USER Kaum cebong kapir udah keliatan dong...   1        1   

   HS_Individual  HS_Group  HS_Religion  HS_Race  HS_Physical  HS_Gender  \
0              1         0            0        0            0          0   
1              0         0            0        0            0          0   
2              0         0            0        0            0          0   
3              0         0            0        0            0          0   
4              0         1            1        0            0          0   

   HS_Other  HS_Weak  HS_Moderate  HS_Strong  
0         1   

In [3]:
# import library penting (anggap sebelumnya sudah training model)
import pandas as pd

# Contoh kalimat baru yang ingin kita prediksi labelnya
sample_texts = [
    "Saya sangat tidak suka dengan perlakuan ini, ini adalah kebencian ras!",
    "Ayo kita dukung perdamaian dan toleransi antar umat beragama",
    "Orang itu kasar dan menyerang secara fisik",
    "Ini hanyalah pendapat saya, tidak bermaksud menghina siapapun"
]

# Transform teks baru pakai vectorizer yang sudah fit sebelumnya
X_new_tfidf = vectorizer.transform(sample_texts)

# Mendapatkan probabilitas prediksi untuk teks baru (hanya untuk RandomForestClassifier yang mendukung)
y_new_proba = multi_rf.predict_proba(X_new_tfidf)

for i, text in enumerate(sample_texts):
    print(f"\nTeks: {text}")
    print("Probabilitas prediksi per label:")
    for label, prob in zip(y.columns, [prob[i][1] for prob in y_new_proba]):
        print(f"  {label}: {prob:.3f}")




Teks: Saya sangat tidak suka dengan perlakuan ini, ini adalah kebencian ras!
Probabilitas prediksi per label:
  HS: 0.160
  Abusive: 0.210
  HS_Individual: 0.140
  HS_Group: 0.380
  HS_Religion: 0.010
  HS_Race: 0.110
  HS_Physical: 0.000
  HS_Gender: 0.000
  HS_Other: 0.090
  HS_Weak: 0.140
  HS_Moderate: 0.350
  HS_Strong: 0.000

Teks: Ayo kita dukung perdamaian dan toleransi antar umat beragama
Probabilitas prediksi per label:
  HS: 0.130
  Abusive: 0.010
  HS_Individual: 0.100
  HS_Group: 0.090
  HS_Religion: 0.140
  HS_Race: 0.000
  HS_Physical: 0.010
  HS_Gender: 0.000
  HS_Other: 0.050
  HS_Weak: 0.030
  HS_Moderate: 0.010
  HS_Strong: 0.040

Teks: Orang itu kasar dan menyerang secara fisik
Probabilitas prediksi per label:
  HS: 0.130
  Abusive: 0.070
  HS_Individual: 0.170
  HS_Group: 0.010
  HS_Religion: 0.080
  HS_Race: 0.000
  HS_Physical: 0.000
  HS_Gender: 0.050
  HS_Other: 0.190
  HS_Weak: 0.150
  HS_Moderate: 0.040
  HS_Strong: 0.000

Teks: Ini hanyalah pendapat saya, t